# --- Day 15: Lens Library ---

https://adventofcode.com/2023/day/15

## Parse the Input Data

In [2]:
def parse(filename):
    """Parse input data for puzzle.

    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.

    Returns
    -------
    initialization_seq : str
        Contains comma separated string of instructions that
        need to be hashed and summed.
    """
    with open(f'../inputs/{filename}.txt') as f:
        initialization_seq = f.read().strip()

    return initialization_seq

In [3]:
parse("initialization_seq")[-50:]

'qb-,xhq=6,zkr-,skhhqs-,bk-,bkj-,ll=9,pqrpll-,krv=3'

## Part 1
---

In [4]:
def _hash(s):
    """Create a hash of a string.

    Parameters
    ----------
    s :str

    Returns
    -------
    curr_val : int
        A hash of the string, s
    """
    curr_val = 0

    for c in s:
        curr_val += ord(c)
        curr_val *= 17
        curr_val %= 256

    return curr_val

In [5]:
def solve1(s):
    return sum([_hash(c) for c in s.split(",")])

### Run on Test Data

In [6]:
_hash("HASH") == 52

True

In [7]:
_hash("rn=1") == 30

True

In [8]:
solve1(parse("test_initialization_seq")) == 1320

True

### Run on Input Data

In [9]:
solve1(parse("initialization_seq"))

503487

## Part 2
---

In [10]:
import math
from collections import defaultdict

In [27]:
def solve2(s):
    boxes = defaultdict(list)
    lenses = {}

    for step in s.split(","):
        if "=" in step:
            label, focal_len = step.split("=")
            box = _hash(label)
            if label not in boxes[box]:
                boxes[box].append(label)
                slot = len(boxes[box])
                lenses[label] = [box + 1, slot, int(focal_len)]
            else:
                slot = boxes[box].index(label) + 1
                lenses[label] = [box + 1, slot, int(focal_len)]

        else:
            label = step[:-1]
            box = _hash(label)
            if label in boxes[box]:
                i_label = boxes[box].index(label)
                boxes[box] = boxes[box][:i_label] + boxes[box][i_label+1:]
                del lenses[label]

                # Adjust slot values down for remaining lenses in the box
                # NOTE: slice from i_label: NOT i_label+1, because I just
                # removed/modified the boxes[box] list above!ß
                for label in boxes[box][i_label:]:
                    lenses[label][1] = lenses[label][1] - 1

    answer = 0
    for val in lenses.values():
        answer += math.prod(val)

    return answer

### Run on Test Data

In [28]:
solve2(parse("test_initialization_seq")) == 145

True

### Run on Input Data

In [29]:
solve2(parse("initialization_seq"))

261505

## Debuggin'

I originally wasn't getting the correct answer, and I couldn't figure out why.

I went here for help first:  
https://www.reddit.com/r/adventofcode/comments/18itgq1/2023_day_15_part_2_help_test_input_wanted/

But my solution passed all the additional tests...

Then the kind help from these folks got me over the hump:  
https://www.reddit.com/r/adventofcode/comments/18jw1a4/2023_day_15_part_2python_help/

In [14]:
solve2("vg=2,ctnb=7,pqfv=1")

112

In [15]:
solve2("rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7,mom=9")

1099

In [16]:
solve2("fdbbhn=1,stdsv=1")

495

In [23]:
_hash("cm") == _hash("rn")

True

In [20]:
solve2("rn=1,cm=2,rn-") #== 2

4

In [24]:
_hash("ot")

3

In [25]:
_hash("ab")

3

In [26]:
_hash("pc")

3